<a href="https://colab.research.google.com/github/adnan1404-ds/ANN-on-Churn-Product-Prediction/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
import kagglehub

# Download latest version of the dataset
path = kagglehub.dataset_download("blastchar/telco-customer-churn")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/telco-customer-churn


In [ ]:
import pandas as pd
import os

# Path from kagglehub
dataset_path = path

# The dataset contains a CSV file, usually named 'WA_Fn-UseC_-Telco-Customer-Churn.csv'
file_path = os.path.join(dataset_path, "WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Read into pandas
df = pd.read_csv(file_path)

print(df.head())


   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

In [ ]:
df['Churn']=df['Churn'].replace(['Yes','No'],[1,0])
#df['Churn']=df['Churn'].map({'Yes':1,'No':0})

/tmp/ipython-input-1921611985.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Churn']=df['Churn'].replace(['Yes','No'],[1,0])


In [ ]:
df.drop('customerID',axis=1,inplace=True)

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

/tmp/ipython-input-266577964.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Convert all object columns to numeric
for col in df.select_dtypes(include=['object']).columns:
    df[col] = le.fit_transform(df[col])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Columns to scale
cols_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges']

scaler = MinMaxScaler()

df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])



In [ ]:
from sklearn.model_selection import train_test_split
X=df.drop('Churn',axis=1)
y=df['Churn']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute weights automatically
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)


Class Weights: {0: np.float64(0.680763653939101), 1: np.float64(1.8830213903743316)}


In [ ]:
X_train.shape


(5634, 19)

In [ ]:
import tensorflow as tf
from keras import layers,models
model=models.Sequential([
    layers.Dense(32,activation='relu',input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(32,activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(16,activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from keras.callbacks import EarlyStopping

# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',   # monitor validation loss
    patience=5,           # stop after 5 epochs with no improvement
    restore_best_weights=True  # roll back to best model
)
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6953 - loss: 0.6022 - val_accuracy: 0.7353 - val_loss: 0.5023
Epoch 2/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7307 - loss: 0.5254 - val_accuracy: 0.7473 - val_loss: 0.4464
Epoch 3/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7389 - loss: 0.4778 - val_accuracy: 0.7885 - val_loss: 0.4424
Epoch 4/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7454 - loss: 0.4792 - val_accuracy: 0.7892 - val_loss: 0.4300
Epoch 5/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7652 - loss: 0.4526 - val_accuracy: 0.7935 - val_loss: 0.4325
Epoch 6/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7749 - loss: 0.4490 - val_accuracy: 0.7977 - val_loss: 0.4283
Epoch 7/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7865 - loss: 0.4483 - val_accuracy: 0.7942 - val_loss: 0.4283
Epoch 8/50
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7844 - loss: 0.4583 - val_accuracy: 0.

In [ ]:
# Evaluate on training set
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)

# Evaluate on testing set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

print("Training Accuracy:", train_acc)
print("Testing Accuracy:", test_acc)
print("Training Loss:", train_loss)
print("Testing Loss:", test_loss)


Training Accuracy: 0.8139865398406982
Testing Accuracy: 0.8090844750404358
Training Loss: 0.4026426672935486
Testing Loss: 0.4084542691707611
